<a href="https://colab.research.google.com/github/LincolnVS/ImersaoDadosAlura/blob/main/Aula_05_Desafios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O notebook da Aula 5 foi separado em duas etapas:
- [Explicações e Aula Alura](https://github.com/LincolnVS/ImersaoDadosAlura/blob/main/Aula_05.ipynb)
- [Desafios e Playground](https://github.com/LincolnVS/ImersaoDadosAlura/blob/main/Aula_05_Desafios.ipynb)

*Obs:* Esse é o notebook de **Desafios e Playground**

### Importando dados

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 15) #mostra apenas 15 colunas quando manda plotar tudo (esse dataset tem mais de 800)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
plt.style.use(style='seaborn-darkgrid')#('fivethirtyeight')
sns.set()

url_dados = 'https://github.com/LincolnVS/ImersaoDadosAlura/blob/main/dados/dados_experimentos.zip?raw=true'

dados = pd.read_csv(url_dados, compression = 'zip')

In [37]:
dados.head()

,id,tratamento,tempo,dose,droga,g-0,g-1,...,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0.5577,...,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0.4087,...,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0.5817,...,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,-0.2491,...,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,-0.4009,...,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [3]:
dados_resultados = pd.read_csv('https://github.com/alura-cursos/imersaodados3/blob/main/dados/dados_resultados.csv?raw=true')
contagem_moa = dados_resultados.drop('id', axis=1).sum().sort_values(ascending=False)
dados_resultados['n_moa'] = dados_resultados.drop('id', axis=1).sum(axis=1)
dados_resultados['ativo_moa'] = (dados_resultados['n_moa'] != 0)
dados_resultados.head()

,id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,...,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,n_moa,ativo_moa
0,id_000644bb2,0,0,0,0,0,0,...,0,0,0,0,0,1,True
1,id_000779bfc,0,0,0,0,0,0,...,0,0,0,0,0,0,False
2,id_000a6266a,0,0,0,0,0,0,...,0,0,0,0,0,3,True
3,id_0015fd391,0,0,0,0,0,0,...,0,0,0,0,0,0,False
4,id_001626bd3,0,0,0,0,0,0,...,0,0,0,0,0,1,True


In [4]:
 dados_combinados = pd.merge(dados, dados_resultados[['id','n_moa', 'ativo_moa']], on='id')
 dados_combinados.head()

,id,tratamento,tempo,dose,droga,g-0,g-1,...,c-95,c-96,c-97,c-98,c-99,n_moa,ativo_moa
0,id_000644bb2,com_droga,24,D1,b68db1d53,1.0620,0.5577,...,0.6584,-0.3981,0.2139,0.3801,0.4176,1,True
1,id_000779bfc,com_droga,72,D1,df89a8e5a,0.0743,0.4087,...,0.4899,0.1522,0.1241,0.6077,0.7371,0,False
2,id_000a6266a,com_droga,48,D1,18bb41b2c,0.6280,0.5817,...,-0.3174,-0.6417,-0.2187,-1.4080,0.6931,3,True
3,id_0015fd391,com_droga,48,D1,8c7f86626,-0.5138,-0.2491,...,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154,0,False
4,id_001626bd3,com_droga,72,D2,7cbed3131,-0.3254,-0.4009,...,-0.3031,0.1094,0.2885,-0.3786,0.7125,1,True


### Dummy model


In [38]:
from sklearn.model_selection import train_test_split

x = dados_combinados.select_dtypes('float64')

y = dados_combinados['ativo_moa'] 
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=376)

          g-0     g-1     g-2     g-3     g-4     g-5     g-6  ...    c-93  \
0      1.0620  0.5577 -0.2479 -0.6208 -0.1944 -1.0120 -1.0220  ...  0.5523   
1      0.0743  0.4087  0.2991  0.0604  1.0190  0.5207  0.2341  ...  0.0230   
2      0.6280  0.5817  1.5540 -0.0764 -0.0323  1.2390  0.1715  ...  0.0223   
3     -0.5138 -0.2491 -0.2656  0.5288  4.0620 -0.8095 -1.9590  ... -1.3780   
4     -0.3254 -0.4009  0.9700  0.6919  1.4180 -0.8244 -0.2800  ...  1.0690   
...       ...     ...     ...     ...     ...     ...     ...  ...     ...   
23809  0.1394 -0.0636 -0.1112 -0.5080 -0.4713  0.7201  0.5773  ...  0.3434   
23810 -1.3260  0.3478 -0.3743  0.9905 -0.7178  0.6621 -0.2252  ... -0.3195   
23811  0.3942  0.3756  0.3109 -0.7389  0.5505 -0.0159 -0.2541  ...  0.2807   
23812  0.6660  0.2324  0.4392  0.2044  0.8531 -0.0343  0.0323  ...  0.1506   
23813 -0.8598  1.0240 -0.1361  0.7952 -0.3611 -3.6750 -1.2420  ... -4.0950   

         c-94    c-95    c-96    c-97    c-98    c-99  
0     -

In [11]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

for option in ["stratified", "most_frequent", "prior", "uniform"]:
  modelo_dummy = DummyClassifier(option)
  modelo_dummy.fit(x_treino, y_treino)
  print(option,modelo_dummy.score(x_teste,y_teste))

stratified 0.52214990552173
most_frequent 0.6067604450976275
prior 0.6067604450976275
uniform 0.5005248792777661


# Desafios 

- *Desafio 01*: Tente outros modelos ou mude os parâmetros dos modelos que testamos em aula.
- *Desafio 02*: Estude a documentação do Sklearn.
- *Desafio 03*: Pesquisar o fluxograma 'Escolhendo o melhor modelo' choosing the right estimator 
- *Desafio 04*: Teste outras problemas (perguntas). 0, 1 ou +.
- *Desafio 05*: Escolha a droga mais utilizada e tente criar um modelo para prever está label. 
- *Desafio 06*: Resumo do que você aprendeu com os dados

## Desafio 1

**Tente outros modelos ou mude os parâmetros dos modelos que testamos em aula.**

In [18]:
from sklearn.linear_model import LogisticRegression
modelo_rlogistica = LogisticRegression(penalty='elasticnet',solver='saga', l1_ratio=0.1,max_iter=2000)
modelo_rlogistica.fit(x_treino, y_treino)
modelo_rlogistica.score(x_teste, y_teste)

0.6269158093638463

In [23]:
modelo_rlogistica = LogisticRegression(max_iter=1000, C=1.5)
modelo_rlogistica.fit(x_treino, y_treino)
modelo_rlogistica.score(x_teste, y_teste)

0.6260760025194205

In [22]:
from sklearn.ensemble import GradientBoostingClassifier
modelo_gbc = GradientBoostingClassifier()

modelo_gbc.fit(x_treino, y_treino)
modelo_gbc.score(x_teste, y_teste)

0.6542095318076843

In [24]:
from sklearn import svm
clf = svm.SVC()
clf.fit(x_treino, y_treino)
clf.score(x_teste, y_teste)

0.6596682762964519

In [26]:
from sklearn.neighbors import KNeighborsClassifier
bagging = KNeighborsClassifier()
bagging.fit(x_treino, y_treino)
bagging.score(x_teste, y_teste)

0.6187276926306949

## Desafio 2

**Estude a documentação do Sklearn.**

https://scikit-learn.org/stable/

## Desafio 3

**Pesquisar o fluxograma 'Escolhendo o melhor modelo' choosing the right estimator**

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

In [27]:
from sklearn import svm 
lin_clf = svm.LinearSVC()
lin_clf.fit(x_treino, y_treino)
lin_clf.score(x_teste, y_teste)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.6048708796976695

## Desafio 4

**Teste outras problemas (perguntas). 0, 1 ou +.**

In [30]:
dados_resultados['ativo_moa_1'] = (dados_resultados['n_moa'] > 1)

dados_resultados['ativo_moa_2'] = (dados_resultados['n_moa'] > 2)

dados_resultados['ativo_moa_3'] = (dados_resultados['n_moa'] > 3)

dados_resultados['ativo_moa_4'] = (dados_resultados['n_moa'] > 3)

In [32]:
x = dados_combinados.select_dtypes('float64')
y = dados_resultados['ativo_moa_1'] 
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=376)

for option in ["stratified", "most_frequent", "prior", "uniform"]:
  modelo_dummy = DummyClassifier(option)
  modelo_dummy.fit(x_treino, y_treino)
  print(option,modelo_dummy.score(x_teste,y_teste))

modelo_rlogistica = LogisticRegression(max_iter=1000, C=1.5)
modelo_rlogistica.fit(x_treino, y_treino)
modelo_rlogistica.score(x_teste, y_teste)  

stratified 0.8545034642032333
most_frequent 0.9195884946462314
prior 0.9195884946462314
uniform 0.5013646861221919


0.9515011547344111

In [33]:
x = dados_combinados.select_dtypes('float64')
y = dados_resultados['ativo_moa_2'] 
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=376)

for option in ["stratified", "most_frequent", "prior", "uniform"]:
  modelo_dummy = DummyClassifier(option)
  modelo_dummy.fit(x_treino, y_treino)
  print(option,modelo_dummy.score(x_teste,y_teste))

modelo_rlogistica = LogisticRegression(max_iter=1000, C=1.5)
modelo_rlogistica.fit(x_treino, y_treino)
modelo_rlogistica.score(x_teste, y_teste)  

stratified 0.9710266638673105
most_frequent 0.9842536216670166
prior 0.9842536216670166
uniform 0.5030442998110435


0.9727062775561621

In [34]:
x = dados_combinados.select_dtypes('float64')
y = dados_resultados['ativo_moa_3'] 
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=376)

for option in ["stratified", "most_frequent", "prior", "uniform"]:
  modelo_dummy = DummyClassifier(option)
  modelo_dummy.fit(x_treino, y_treino)
  print(option,modelo_dummy.score(x_teste,y_teste))

modelo_rlogistica = LogisticRegression(max_iter=1000, C=1.5)
modelo_rlogistica.fit(x_treino, y_treino)
modelo_rlogistica.score(x_teste, y_teste)  

stratified 0.9922317866890615
most_frequent 0.9968507243334033
prior 0.9968507243334033
uniform 0.5001049758555532


0.9960109174889775

In [35]:
x = dados_combinados.select_dtypes('float64')
y = dados_resultados['ativo_moa_4'] 
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=376)

for option in ["stratified", "most_frequent", "prior", "uniform"]:
  modelo_dummy = DummyClassifier(option)
  modelo_dummy.fit(x_treino, y_treino)
  print(option,modelo_dummy.score(x_teste,y_teste))

modelo_rlogistica = LogisticRegression(max_iter=1000, C=1.5)
modelo_rlogistica.fit(x_treino, y_treino)
modelo_rlogistica.score(x_teste, y_teste)  

stratified 0.9913919798446358
most_frequent 0.9968507243334033
prior 0.9968507243334033
uniform 0.49527608650010496


0.9960109174889775

## Desafio 5

**Escolha a droga mais utilizada e tente criar um modelo para prever está label.**

In [39]:
#Saber se é controle ou não 
#(praticamente a mesma coisa que responder se é um tipo de droga ou naõ, pois com controle é a "droga" mais usada)
from sklearn.model_selection import train_test_split

x = dados_combinados.select_dtypes('float64')
y = (dados['tratamento'] == 'com_controle')

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2, stratify=y, random_state=376)

In [42]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

for option in ["stratified", "most_frequent", "prior", "uniform"]:
  modelo_dummy = DummyClassifier(option)
  modelo_dummy.fit(x_treino, y_treino)
  print(option,modelo_dummy.score(x_teste,y_teste))
  
modelo_rlogistica = LogisticRegression(max_iter=1000, C=1.5)
modelo_rlogistica.fit(x_treino, y_treino)
modelo_rlogistica.score(x_teste, y_teste)

stratified 0.8624816292252782
most_frequent 0.9216880117572959
prior 0.9216880117572959
uniform 0.5007348309888726


0.9542305269787948

## Desafio 6

**Resumo do que você aprendeu com os dados**

- Testem de forma prática, a aplicação de modelos de Machine Learning para fazer algumas classificações no dataset
- Variação de modelos podem influenciar nos resultados obtidos.
- Às vezes, conforme os dados “y” estão separados, até modelos mais "burros" conseguem ter uma boa acurácia.